In [1]:
from bs4 import BeautifulSoup as bs
#from sklearn.cluster import KMeans
import requests
from pymongo import MongoClient
import pandas as pd
import time

In [3]:
client = MongoClient('localhost', 27017)
db = client['hw4']
appdata = db['appdata']

In [4]:
content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1")
soup = bs(content.text, 'html.parser')


In [5]:
def getDetails(url):
    #getting up beautifulsoup and requests so we can crawl the url passed to the function
    content = requests.get(url)
    soup = bs(content.text, 'html.parser')
    
    #creating empty list to be filled with a list of details for every apartement
    detlist =[]
    for app in soup.find_all(class_="listing-item vetrina js-row-detail" ):
        #creating empty list to fill with the details
        dets =[]
        
        #Retrieving the link of the apartement thats contained in the class "titolo text-primary"
        #by searching for the first occurence of 'href'.
        link_class = app.find(class_="titolo text-primary")
        link = link_class.find('a', href=True)['href']
        
        #To skip the advertisments
        if link.startswith('/nuove_costruzioni/'):
            continue
        #appending the link to the list of details    
        dets.append(link)
        
        #Retrieving the price of the apartement. It is contained in the second item in the list that beautifulsoup
        price = app.find(class_="listing-features list-piped")
        #appending the price to the list of details
        dets.append(price.contents[1].text.strip().split(" ")[1])

        #getting the information on the other attributes of the appartement (e.g. bagni, piano )
        for details in app.find_all(class_="lif__data"):
            det = details.contents[0]
            dets.append(det.get_text().strip())
            
        #appending the list of details for the current apartement to the list of the
        #details of the other appartement on the current page of the search result
        if len(dets) == 6:
            detlist.append(dets)
    
    return detlist

In [6]:
i = 0
data = pd.DataFrame()
i = 1
while data.shape[0] < 100:
    #setting up the urls to be crawled by our function, starting from page one (as i is defined as 1 befor executing)
    url = "https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i)
    
    #appending the nested list with the lists of details for every apartement to the dataframe
    data = data.append(getDetails(url))

    #increasing i by one so the next page of search results will be crawled
    i+=1
    #setting  timer so our requests won't get blocked
    time.sleep(5)


In [12]:
data = data.reset_index(drop=True)

In [13]:
data

,0,1,2,3,4,5
0,https://www.immobiliare.it/53131931-Vendita-Bi...,225.000,2,50,1,1
1,https://www.immobiliare.it/69489650-Vendita-Qu...,1.350.000,4,200,2,2
2,https://www.immobiliare.it/69192522-Vendita-Qu...,450.000,4,135,1,A
3,https://www.immobiliare.it/68192795-Vendita-At...,135.000,2,60,1,5
4,https://www.immobiliare.it/68597785-Vendita-Bi...,249.000,2,75,1,1
5,https://www.immobiliare.it/68751099-Vendita-Bi...,339.000,2,70,1,3
6,https://www.immobiliare.it/66093203-Vendita-At...,699.000,5+,174,3,A
7,https://www.immobiliare.it/69722980-Vendita-Qu...,189.000,4,168,3+,T
8,https://www.immobiliare.it/70265150-Vendita-Tr...,215.000,3,67,1,1
9,https://www.immobiliare.it/70107928-Vendita-Tr...,279.000,3,110,1,T


In [14]:
data.to_csv('df.csv', sep='\t')